In [1]:
import math
import decimal as dec
from decimal import Decimal
import numpy as np
import pandas as pd

In [2]:
def f1(x):
    return Decimal(math.cos(x)) * Decimal(math.cosh(x)) - 1

def f1_bounds():
    return (3 / 2 * math.pi, 2 * math.pi)

def f1_deriv(x):
    return Decimal(math.cos(x)) * Decimal(math.sinh(x)) - Decimal(math.sin(x)) * Decimal(math.cosh(x))


def f2(x):
    # To avoid numerical problems
    if math.isclose(x, 0, abs_tol = 1e-9): 
        return Decimal(10000000)
    return Decimal(1 / x) - Decimal(math.tan(x))

def f2_bounds():
    return (0, math.pi / 2)

def f2_deriv(x):
    return Decimal((-1) / x ** 2) - Decimal(1 / math.cos(x) ** 2)


def f3(x):
    return Decimal(2 ** (-x)) + Decimal(math.exp(x)) + Decimal((2 * math.cos(x) - 6))

def f3_bounds():
    return (1, 3)

def f3_deriv(x):
    return Decimal(math.exp(x)) - Decimal(2 ** (-x)) * Decimal(math.log(2, math.e)) - Decimal(2 * math.sin(x))

Well for large x solution to (f1 = 0) will be approximately cos(x) = 0 <=> x = (n + 1/2) * PI. 
It's due to the fact that cosh(x) grows exponentially.

In [3]:
class Output(object):
    def __init__(self, x0, iterations):
        self.x0 = x0
        self.iterations = iterations

def bisection(f, digits, accuracy, a, b):
    dec.getcontext().prec = digits
    iterations = 0
    a, b = Decimal(a), Decimal(b)
    
    while True:
        iterations += 1
        c = (a + b) / 2
        fc = f(c)
        if math.isclose(fc, 0, abs_tol = accuracy) or math.isclose((b - a) / 2, 0, abs_tol = accuracy):
            return Output(c, iterations)
        
        fa = f(a)
        if (fc < 0 and fa < 0) or (fc > 0 and fa > 0):
            a = c
        else:
            b = c

In [4]:
def newton(f, f_deriv, digits, max_iterations, accuracy, a, b):
    dec.getcontext().prec = digits
    a, b = Decimal(a), Decimal(b)
    
    x = (a + b) / 2 # Initial guess
    n = 0
    while True:
        fx = f(x)
        if math.isclose(fx, 0, abs_tol = accuracy) or n >= max_iterations:
            return Output(x, n)
        n += 1
        x -= f(x) / f_deriv(x)

In [5]:
def secant(f, digits, max_iterations, accuracy, a, b):
    dec.getcontext().prec = digits
    # Two arbitrarily chosen values to start with.   
    a, b = Decimal(a + 1), Decimal(b + 1)
    n = 0
    
    while True:
        fa = f(a)
        fb = f(b)
        
        # If |fa - fb| yields 0 we finish procedure.
        if math.isclose(fa, 0, abs_tol = accuracy) or n >= max_iterations or math.isclose(fa - fb, 0, abs_tol= acc):
            return Output(a, n)
        
        n += 1
        x = (b * fa - a * fb) / (fa - fb)
        b, a = a, x

In [6]:
### Testing
def append(df, function, method, accuracy, output):
    row = {
        'Function': function,
        'Method': method,
        'Accuracy': accuracy,
        'Iterations': output.iterations,
        'Root': output.x0
    }
    
    series = pd.Series(data = row, name = 'x')
    return df.append(series, ignore_index = False)

df = pd.DataFrame(columns = ['Function', 'Method', 'Accuracy', 'Iterations', 'Root'])
max_iter = 124

for x in [7, 15, 31]:
    acc = 10 ** (-x)
    acc_str = 'E-' + str(x)
    bisection_out = bisection(f1, x + 3, acc, *(f1_bounds()))
    df = append(df, 'f1', 'bisection', acc_str, bisection_out)
    
    newton_out = newton(f1, f1_deriv, x + 3, max_iter, acc, *(f1_bounds()))
    df = append(df, 'f1', 'newton', acc_str, newton_out)
    
    secant_out = secant(f1, x + 3, max_iter, acc, *(f1_bounds()))
    df = append(df, 'f1', 'secant', acc_str, secant_out)

    bisection_out_f2 = bisection(f2, x + 3, acc, *(f2_bounds()))
    df = append(df, 'f2', 'bisection', acc_str, bisection_out_f2)
    
    newton_out_f2 = newton(f2, f2_deriv, x + 3, max_iter, acc, *(f2_bounds()))
    df = append(df, 'f2', 'newton', acc_str, newton_out_f2)
    
    secant_out_f2 = secant(f2, x + 3, max_iter, acc, *(f2_bounds()))
    df = append(df, 'f2', 'secant', acc_str, secant_out_f2)
    
    bisection_out_f3 = bisection(f3, x + 3, acc, *(f3_bounds()))
    df = append(df, 'f3', 'bisection', acc_str, bisection_out_f3)
    
    newton_out_f3 = newton(f3, f3_deriv, x + 3, max_iter, acc, *(f3_bounds()))
    df = append(df, 'f3', 'newton', acc_str, newton_out_f3)
    
    secant_out_f3 = secant(f3, x + 3, max_iter, acc, *(f3_bounds()))
    df = append(df, 'f3', 'secant', acc_str, secant_out_f3)
    
    
df

,Function,Method,Accuracy,Iterations,Root
x,f1,bisection,E-7,24,4.730040714
x,f1,newton,E-7,5,4.730040745
x,f1,secant,E-7,8,4.730040745
x,f2,bisection,E-7,24,0.860333556
x,f2,newton,E-7,3,0.8603335890
x,f2,secant,E-7,7,0.8603335896
x,f3,bisection,E-7,23,1.829383612
x,f3,newton,E-7,4,1.829383602
x,f3,secant,E-7,6,1.829383601
x,f1,bisection,E-15,51,4.73004074486270388


## Comparison

Newton's method outperforms the bisection method when it comes to speed, although it's sensitive to starting point and it can get locked into a periodic iteration never reaching requied accuracy. 
Given correct interval convergence is sure with bisection method. Secant method is finite-difference approximation of Newton's method (we approximate the derivtative which is given to us in Newton
s method) as such it's vulnerable to the same problems as Newton's method + error from finite-difference approximation.  